 **packages that are used in this file**

In [1]:
!pip install gradientai --upgrade
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.1/171.1 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.6/137.6 kB 10.8 MB/s eta 0:00:00


In [ ]:
import os
from langchain.chains import LLMChain
from langchain.llms import GradientLLM
from langchain.prompts import PromptTemplate

**Set the environment variables for gradient.ai using your access token and workspace ID**


In [6]:
# Set the environment variables for gradient.ai using your access token and workspace ID
os.environ['GRADIENT_ACCESS_TOKEN'] = "1vhvNKf2lLAMPvrOqhV97xaPPNRzwT1J"
os.environ['GRADIENT_WORKSPACE_ID'] = "b04a475d-65d1-4e98-82f2-62d218be3989_workspace"

**This section loads the dataset from a JSON file and divides it into smaller chunks for fine-tuning the model**

In [4]:
def divide_into_chunks(number, chunk_size): # Define a function to divide a number into chunks of a given size
    chunks = []
    while number > 0:
        if number >= chunk_size:
            chunks.append(chunk_size)
            number -= chunk_size
        else:
            chunks.append(number)
            break

    return chunks

with open("/content/data.json") as f:
        samples = json.load(f)

chunks = divide_into_chunks(len(samples), 100)
print("dataset chunks : ", chunks)


IndentationError: ignored

**# This section fine-tunes a model on the gradient.ai platform using the dataset**


In [11]:
from gradientai import Gradient

with Gradient() as gradient:
    base_model = gradient.get_base_model(base_model_slug="nous-hermes2")
    new_model_adapter = base_model.create_model_adapter(name="Animal_husbandry-LLM(hezron)"  )
    print(f"Created model adapter with id {new_model_adapter.id}")

    num_epochs = 3 # num_epochs is the number of times you fine-tune the model # more epochs tends to get better results, but you also run the risk of "overfitting"
    count = 0
    while count < num_epochs:
        print(f"Fine-tuning the model, iteration {count + 1}")
        s = 0
        n = 1
        for i in chunks:
            print(f"chunk {n} range: {s} : {(s+i)}")
            while True:
              try:
                new_model_adapter.fine_tune(samples=samples[s: (s+i)])
                break
              except:
                pass
            s += i
            n += 1
        count = count + 1

#new_model_adapter.delete()


Created model adapter with id 9051b37c-f422-43e1-92db-bfed72adf1fb_model_adapter
[100, 100, 100, 100, 100, 100, 100, 100, 39]
Fine-tuning the model, iteration 1
chunk 1 range: 0 : 100


KeyboardInterrupt: ignored

**# This section tests the fine-tuned model**

In [7]:


model_id = new_model_adapter.id
llm = GradientLLM(
    model="4f2d6d3c-4685-493d-b945-08da7f120021_model_adapter",
    model_kwargs=dict(max_generated_token_count=128),
)

In [ ]:
template = """### Instruction: {Instruction} \n\n### Response:"""

prompt = PromptTemplate(template=template, input_variables=["Instruction"])
llm_chain = LLMChain(prompt=prompt, llm=llm)

In [ ]:
Question = "Discuss the role of nutrition in animal husbandry"
llm_chain.run(Instruction=Question)

' Nutrition plays a vital role in animal husbandry, ensuring optimal health and productivity. Proper nutrition involves balancing diets with essential nutrients, considering factors such as age, breed, and environmental conditions. A well-balanced diet enhances overall health, supports reproductive performance, and contributes to the quality of dairy products. Regular monitoring and adjustments to nutritional requirements are essential for optimizing nutrition in animal husbandry.'

# Retrieval Augmented Generation Gradient Embeddings and Generators```
**bold text**


In [ ]:
!pip install -U gradient_haystack

In [ ]:
from gradient_haystack.embedders.gradient_document_embedder import GradientDocumentEmbedder
from gradient_haystack.embedders.gradient_text_embedder import GradientTextEmbedder
from gradient_haystack.generator.base import GradientGenerator
from haystack import Document, Pipeline
from haystack.components.writers import DocumentWriter
from haystack.document_stores import InMemoryDocumentStore
from haystack.components.retrievers import InMemoryEmbeddingRetriever
from haystack.components.builders import PromptBuilder
from haystack.components.builders.answer_builder import AnswerBuilder
import os

In [ ]:
os.environ['GRADIENT_ACCESS_TOKEN'] = "1vhvNKf2lLAMPvrOqhV97xaPPNRzwT1J"
os.environ['GRADIENT_WORKSPACE_ID'] = "b04a475d-65d1-4e98-82f2-62d218be3989_workspace"

In [ ]:
with open("/content/Raw_Text_Data.txt", 'r') as file:
    text_data = file.read()

docs = [
        Document(content=text_data)
        ]


In [ ]:
document_embedder = GradientDocumentEmbedder(
    access_token=os.environ["GRADIENT_ACCESS_TOKEN"],
    workspace_id=os.environ["GRADIENT_WORKSPACE_ID"],
)


text_embedder = GradientTextEmbedder(
    access_token=os.environ["GRADIENT_ACCESS_TOKEN"],
    workspace_id=os.environ["GRADIENT_WORKSPACE_ID"],
)

generator = GradientGenerator(
    access_token=os.environ["GRADIENT_ACCESS_TOKEN"],
    workspace_id=os.environ["GRADIENT_WORKSPACE_ID"],
    base_model_slug="llama2-7b-chat",
    max_generated_token_count=350,
)




In [ ]:
indexing_pipeline = Pipeline()
indexing_pipeline.add_component(instance=document_embedder, name="document_embedder")
indexing_pipeline.add_component(instance=writer, name="writer")
indexing_pipeline.connect("document_embedder", "writer")
indexing_pipeline.run({"document_embedder": {"documents": docs}})

In [ ]:
prompt = """You are helpful assistant ment to answer questions relating to animal husbandry. Answer the query, based on the
content in the documents. if you dont know the answer say you don't know.
{{documents}}
Query: {{query}}
\nAnswer:
"""

**. Building RAG Pipeline:**

In [ ]:
retriever = InMemoryEmbeddingRetriever(document_store=document_store)
prompt_builder = PromptBuilder(template=prompt)

rag_pipeline = Pipeline()
rag_pipeline.add_component(instance=text_embedder, name="text_embedder")
rag_pipeline.add_component(instance=retriever, name="retriever")
rag_pipeline.add_component(instance=prompt_builder, name="prompt_builder")
rag_pipeline.add_component(instance=generator, name="generator")
rag_pipeline.add_component(instance=AnswerBuilder(), name="answer_builder")
rag_pipeline.connect("generator.replies", "answer_builder.replies")
rag_pipeline.connect("retriever", "answer_builder.documents")
rag_pipeline.connect("text_embedder", "retriever")
rag_pipeline.connect("retriever", "prompt_builder.documents")
rag_pipeline.connect("prompt_builder", "generator")


In [ ]:
def LLM_Run(question):
    result = rag_pipeline.run(
        {
            "text_embedder": {"text": question},
            "prompt_builder": {"query": question},
            "answer_builder": {"query": question}
        }
    )
    return result["answer_builder"]["answers"][0].data

In [ ]:
Query = "What is animal husbandry??"
print(LLM_Run(Query))
